In [1]:
import numpy as np
import pandas as pd
import scanpy
import cell2sentence
import torch
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer
from datasets import load_dataset, Dataset
import evaluate
import nltk
import random
import pickle

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\johnz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
eso = scanpy.read_h5ad("../data/TabulaSapiens.h5ad")

In [4]:
eso.obs

,tissue_in_publication,assay_ontology_term_id,donor_id,anatomical_information,n_counts_UMIs,n_genes,cell_ontology_class,free_annotation,manually_annotated,compartment,...,tissue_ontology_term_id,suspension_type,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage
cell_id,,,,,,,,,,,,,,,,,,,,,
AAACCCACACTCCTGT_TSP6_Liver_NA_10X_1_1,Liver,EFO:0009922,TSP6,nan,7633.0,2259,macrophage,Monocyte/Macrophage,True,immune,...,UBERON:0002107,cell,macrophage,10x 3' v3,normal,Homo sapiens,male,liver,European,67-year-old human stage
AAACGAAGTACCAGAG_TSP6_Liver_NA_10X_1_1,Liver,EFO:0009922,TSP6,nan,2858.0,1152,monocyte,Monocyte,True,immune,...,UBERON:0002107,cell,monocyte,10x 3' v3,normal,Homo sapiens,male,liver,European,67-year-old human stage
AAACGCTCAACGGCTC_TSP6_Liver_NA_10X_1_1,Liver,EFO:0009922,TSP6,nan,7787.0,2983,endothelial cell of hepatic sinusoid,Endothelial,True,endothelial,...,UBERON:0002107,cell,endothelial cell of hepatic sinusoid,10x 3' v3,normal,Homo sapiens,male,liver,European,67-year-old human stage
AAAGAACAGCCTCTTC_TSP6_Liver_NA_10X_1_1,Liver,EFO:0009922,TSP6,nan,10395.0,2598,macrophage,Monocyte/Macrophage,True,immune,...,UBERON:0002107,cell,macrophage,10x 3' v3,normal,Homo sapiens,male,liver,European,67-year-old human stage
AAAGAACGTAGCACAG_TSP6_Liver_NA_10X_1_1,Liver,EFO:0009922,TSP6,nan,6610.0,2125,liver dendritic cell,Dendritic cell,True,immune,...,UBERON:0002107,cell,liver dendritic cell,10x 3' v3,normal,Homo sapiens,male,liver,European,67-year-old human stage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TSP2_Vasculature_aorta_SS2_B114577_B133059_Endothelial_P4_S364,Vasculature,EFO:0008931,TSP2,aorta,13205.0,579,endothelial cell,endothelial cell,True,endothelial,...,UBERON:0000947,cell,endothelial cell,Smart-seq2,normal,Homo sapiens,female,aorta,African American or Afro-Caribbean,61-year-old human stage
TSP2_Vasculature_aorta_SS2_B114577_B133059_Endothelial_P5_S365,Vasculature,EFO:0008931,TSP2,aorta,9565.0,529,endothelial cell,endothelial cell,True,endothelial,...,UBERON:0000947,cell,endothelial cell,Smart-seq2,normal,Homo sapiens,female,aorta,African American or Afro-Caribbean,61-year-old human stage
TSP2_Vasculature_aorta_SS2_B114577_B133059_Endothelial_P7_S367,Vasculature,EFO:0008931,TSP2,aorta,195639.0,2753,endothelial cell,endothelial cell,True,endothelial,...,UBERON:0000947,cell,endothelial cell,Smart-seq2,normal,Homo sapiens,female,aorta,African American or Afro-Caribbean,61-year-old human stage


In [3]:
eso_c2s = cell2sentence.transforms.csdata_from_adata(eso)

100%|██████████| 483152/483152 [05:09<00:00, 1562.13it/s]


In [4]:
cell_sents = eso_c2s.create_sentence_lists()

In [5]:
labels = eso.obs['cell_type'].values
labels = [str(label) for label in labels]

In [8]:
label2id = dict(enumerate(np.unique(labels)))

In [6]:
del(eso, eso_c2s)

In [10]:
#cell_sents = [inner_list[:300] for inner_list in cell_sents]

In [7]:
cell_sents = [' '.join(inner_list) for inner_list in cell_sents]

In [8]:
# Create a pandas DataFrame with the sentences and labels
data = {"text": cell_sents, "cell_type": labels}

In [9]:
del(cell_sents, labels)

In [10]:
# Create the dataset from the pandas DataFrame
dataset = Dataset.from_dict(data)

In [11]:
del(data)

In [ ]:
dataset = dataset.train_test_split(test_size=0.05)

In [16]:
dataset['train']

Dataset({
    features: ['text', 'cell_type'],
    num_rows: 458994
})

In [17]:
dataset['train'].to_json("../data/TabulaSapiens_raw_dataset_train.json")

Creating json from Arrow format:   0%|          | 0/459 [00:00<?, ?ba/s]

19332838191

In [18]:
dataset['test'].to_json("../data/TabulaSapiens_raw_dataset_test.json")

Creating json from Arrow format:   0%|          | 0/25 [00:00<?, ?ba/s]

1015088121

In [ ]:
dataset = load_dataset('json', data_files={'train': '../data/TabulaSapiens_raw_dataset_train.json','test': '../data/TabulaSapiens_raw_dataset_test.json"'})

In [16]:
dataset

Dataset({
    features: ['text', 'cell_type'],
    num_rows: 483152
})

In [2]:
#with open('../data/TabulaSapiens_raw_dataset.pkl', 'wb') as f:
#    pickle.dump(dataset, f)

In [1]:
#dataset = pickle.load(open('../data/TabulaSapiens_raw_dataset.pkl', 'rb'))

In [2]:
prefix = "label cell type: "
max_input_length = 512
max_target_length = 64

tokenizer = AutoTokenizer.from_pretrained("t5-base")

def preprocess_data(examples):
  inputs = [prefix + text for text in examples["text"]]
  model_inputs = tokenizer(inputs, max_length=max_input_length, padding=True, truncation=True)

  # Setup the tokenizer for targets
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples["cell_type"], max_length=max_target_length, padding=True, truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

c:\Users\johnz\anaconda3\envs\celldl\lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [16]:
dataset

Dataset({
    features: ['text', 'cell_type'],
    num_rows: 87947
})

In [20]:
dataset['train'][0]

{'text': 'ENSG00000245975 ENSG00000264112 ENSG00000213305 ENSG00000129682 ENSG00000241535 ENSG00000272717 ENSG00000239827 ENSG00000212195 ENSG00000198618 ENSG00000148735 ENSG00000199568 ENSG00000273254 ENSG00000246100 ENSG00000167333 ENSG00000272750 ENSG00000243725 ENSG00000272908 ENSG00000286071 ENSG00000249936 ENSG00000050628 ENSG00000280088 ENSG00000285646 ENSG00000269313 ENSG00000267278 ENSG00000154548 ENSG00000285725 ENSG00000185480 ENSG00000233230 ENSG00000243406 ENSG00000263069 ENSG00000266962 ENSG00000174032 ENSG00000279529 ENSG00000177103 ENSG00000200156 ENSG00000211813 ENSG00000267254 ENSG00000260318 ENSG00000233328 ENSG00000274156 ENSG00000261586 ENSG00000180815 ENSG00000174876 ENSG00000272370 ENSG00000013619 ENSG00000286268 ENSG00000187733 ENSG00000109881 ENSG00000165238 ENSG00000271856 ENSG00000171574 ENSG00000166446 ENSG00000065491 ENSG00000189114 ENSG00000075336 ENSG00000165046 ENSG00000196437 ENSG00000137976 ENSG00000127081 ENSG00000187533 ENSG00000236778 ENSG0000016361

In [21]:
tokenized_datasets = dataset.map(preprocess_data,batched=True)

Map:   0%|          | 0/458994 [00:00<?, ? examples/s]

c:\Users\johnz\anaconda3\envs\celldl\lib\site-packages\transformers\tokenization_utils_base.py:3586: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/24158 [00:00<?, ? examples/s]

In [22]:
tokenized_datasets['train'].to_json("../data/TabulaSapiens_tokenized_dataset_train.json")

Creating json from Arrow format:   0%|          | 0/459 [00:00<?, ?ba/s]

20949452142

In [23]:
tokenized_datasets['test'].to_json("../data/TabulaSapiens_tokenized_dataset_test.json")

Creating json from Arrow format:   0%|          | 0/25 [00:00<?, ?ba/s]

1100173743

In [3]:
tokenized_datasets = load_dataset('json', data_files={'train': '../data/TabulaSapiens_tokenized_dataset_train.json','test': '../data/TabulaSapiens_tokenized_dataset_test.json'})

Found cached dataset json (C:/Users/johnz/.cache/huggingface/datasets/json/default-3fcedefba4e19bbd/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


  0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
#with open('data/tokenized_datasets.pkl', 'wb') as f:
#    pickle.dump(tokenized_datasets, f)

In [3]:
#tokenized_datasets = pickle.load(open('data/tokenized_datasets.pkl', 'rb'))

In [4]:
batch_size = 4
args = Seq2SeqTrainingArguments(
    output_dir='t5',
    evaluation_strategy="steps",
    eval_steps=10000,
    logging_strategy="steps",
    logging_steps=200,
    save_strategy="steps",
    save_steps=10000,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=False,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    report_to="tensorboard"
)

data_collator = DataCollatorForSeq2Seq(tokenizer)
metric = evaluate.load("rouge")

In [5]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) 
                      for label in decoded_labels]
    
    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value * 100 for key, value in result.items()}
    
    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [6]:
def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained('t5-base')

In [7]:
trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [8]:
trainer.train()

c:\Users\johnz\anaconda3\envs\celldl\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/114749 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 1.0963, 'learning_rate': 3.993028261684198e-05, 'epoch': 0.0}
{'loss': 0.3335, 'learning_rate': 3.9860565233683956e-05, 'epoch': 0.0}
{'loss': 0.2864, 'learning_rate': 3.9790847850525935e-05, 'epoch': 0.01}
{'loss': 0.2502, 'learning_rate': 3.972113046736791e-05, 'epoch': 0.01}
{'loss': 0.2309, 'learning_rate': 3.965141308420989e-05, 'epoch': 0.01}
{'loss': 0.2322, 'learning_rate': 3.958169570105186e-05, 'epoch': 0.01}
{'loss': 0.2238, 'learning_rate': 3.951197831789384e-05, 'epoch': 0.01}
{'loss': 0.218, 'learning_rate': 3.944226093473581e-05, 'epoch': 0.01}
{'loss': 0.2166, 'learning_rate': 3.937254355157779e-05, 'epoch': 0.02}
{'loss': 0.2141, 'learning_rate': 3.930282616841977e-05, 'epoch': 0.02}
{'loss': 0.2129, 'learning_rate': 3.9233108785261746e-05, 'epoch': 0.02}
{'loss': 0.2083, 'learning_rate': 3.9163391402103725e-05, 'epoch': 0.02}
{'loss': 0.2008, 'learning_rate': 3.9093674018945705e-05, 'epoch': 0.02}
{'loss': 0.1987, 'learning_rate': 3.902395663578768e-05, 'epoc

  0%|          | 0/6040 [00:00<?, ?it/s]

...

  0%|          | 0/6040 [00:00<?, ?it/s]

{'eval_loss': 0.05219770595431328, 'eval_rouge1': 79.9518, 'eval_rouge2': 52.1942, 'eval_rougeL': 79.9293, 'eval_rougeLsum': 79.9205, 'eval_gen_len': 7.0773, 'eval_runtime': 2885.9647, 'eval_samples_per_second': 8.371, 'eval_steps_per_second': 2.093, 'epoch': 0.87}
{'loss': 0.0573, 'learning_rate': 5.071591037830396e-06, 'epoch': 0.87}
{'loss': 0.0569, 'learning_rate': 5.001873654672372e-06, 'epoch': 0.87}
{'loss': 0.0603, 'learning_rate': 4.932156271514349e-06, 'epoch': 0.88}
{'loss': 0.062, 'learning_rate': 4.862438888356326e-06, 'epoch': 0.88}
{'loss': 0.0586, 'learning_rate': 4.7927215051983025e-06, 'epoch': 0.88}
{'loss': 0.0584, 'learning_rate': 4.72300412204028e-06, 'epoch': 0.88}
{'loss': 0.0558, 'learning_rate': 4.653286738882257e-06, 'epoch': 0.88}
{'loss': 0.0545, 'learning_rate': 4.583569355724233e-06, 'epoch': 0.89}
{'loss': 0.062, 'learning_rate': 4.51385197256621e-06, 'epoch': 0.89}
{'loss': 0.0557, 'learning_rate': 4.444134589408186e-06, 'epoch': 0.89}
{'loss': 0.0572, 

  0%|          | 0/6040 [00:00<?, ?it/s]

{'eval_loss': 0.05088723078370094, 'eval_rouge1': 80.2109, 'eval_rouge2': 52.1682, 'eval_rougeL': 80.193, 'eval_rougeLsum': 80.1796, 'eval_gen_len': 7.0882, 'eval_runtime': 2711.379, 'eval_samples_per_second': 8.91, 'eval_steps_per_second': 2.228, 'epoch': 0.96}
{'loss': 0.0574, 'learning_rate': 1.585721879929237e-06, 'epoch': 0.96}
{'loss': 0.0571, 'learning_rate': 1.5160044967712137e-06, 'epoch': 0.96}
{'loss': 0.0618, 'learning_rate': 1.4462871136131906e-06, 'epoch': 0.96}
{'loss': 0.0578, 'learning_rate': 1.3765697304551675e-06, 'epoch': 0.97}
{'loss': 0.0605, 'learning_rate': 1.3068523472971442e-06, 'epoch': 0.97}
{'loss': 0.0544, 'learning_rate': 1.2371349641391211e-06, 'epoch': 0.97}
{'loss': 0.0578, 'learning_rate': 1.167417580981098e-06, 'epoch': 0.97}
{'loss': 0.0569, 'learning_rate': 1.0977001978230747e-06, 'epoch': 0.97}
{'loss': 0.0602, 'learning_rate': 1.0279828146650516e-06, 'epoch': 0.97}
{'loss': 0.0538, 'learning_rate': 9.582654315070285e-07, 'epoch': 0.98}
{'loss': 0

TrainOutput(global_step=114749, training_loss=0.08713308761251343, metrics={'train_runtime': 76963.6092, 'train_samples_per_second': 5.964, 'train_steps_per_second': 1.491, 'train_loss': 0.08713308761251343, 'epoch': 1.0})

In [9]:
trainer.evaluate()

  0%|          | 0/6040 [00:00<?, ?it/s]

{'eval_loss': 0.05088723078370094,
 'eval_rouge1': 80.2109,
 'eval_rouge2': 52.1682,
 'eval_rougeL': 80.193,
 'eval_rougeLsum': 80.1796,
 'eval_gen_len': 7.0882,
 'eval_runtime': 2619.9773,
 'eval_samples_per_second': 9.221,
 'eval_steps_per_second': 2.305,
 'epoch': 1.0}

In [13]:
trainer.save_model('t5/Tabula5apiens-1epoch')

In [9]:
tokenizer = AutoTokenizer.from_pretrained('t5/checkpoint-52000/')
model = AutoModelForSeq2SeqLM.from_pretrained('t5/checkpoint-52000/')

In [10]:
model.to('cuda')

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [11]:
max_input_length = 512
texts = dataset['test']['text'][39]
#inputs = [prefix + text for text in texts]
inputs = [prefix + texts]
inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt").to('cuda')
output = model.generate(**inputs, num_beams=32, do_sample=True, min_length=0, max_length=64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_title)

NameError: name 'dataset' is not defined

In [ ]:
dataset['test']['cell_type'][39]

'CD8-positive, alpha-beta cytotoxic T cell'

In [ ]:
for x in random.sample(range(0, len(dataset['test']['text'])),25):
    max_input_length = 512
    texts = dataset['test']['text'][x]
    #inputs = [prefix + text for text in texts]
    inputs = [prefix + texts]
    inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt").to('cuda')
    output = model.generate(**inputs, num_beams=32, do_sample=True, min_length=0, max_length=64)
    decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

    print(str(x) + ': ' + predicted_title + ' - ' + dataset['test']['cell_type'][x])

9722: epithelial cell - epithelial cell
7133: stratified epithelial cell - epithelial cell
1916: stratified epithelial cell - stratified epithelial cell
1997: CD8-positive, alpha-beta T cell - stromal cell
10276: epithelial cell - epithelial cell
1873: stratified epithelial cell - stratified epithelial cell
1643: epithelial cell - epithelial cell
15623: epithelial cell - epithelial cell
16477: stratified epithelial cell - stratified epithelial cell
17403: stratified epithelial cell - stratified epithelial cell
5158: stratified epithelial cell - stratified epithelial cell
1863: stratified epithelial cell - stratified epithelial cell
16640: stratified epithelial cell - stratified epithelial cell
2625: epithelial cell - epithelial cell
6089: stratified epithelial cell - stratified epithelial cell
2245: stratified epithelial cell - stratified epithelial cell
2226: stratified epithelial cell - stratified epithelial cell
7707: epithelial cell - epithelial cell
13230: epithelial cell - epithe